In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt 

tweets = pd.read_csv('combined.csv', encoding='utf-8')
tweets.head(5)

,tweet_date_created,tweet_id,tweet_text,language,sentiment,sentiment_score
0,2018-05-08T08:19:09,993767246437666816,Bayer Leverkusen goalkeeper Bernd Leno will no...,en,NEUTRAL,"{""Neutral"":0.7228581905364990234375,""Negative""..."
1,2018-07-02T19:28:00.331000,1013866900772835331,Gary Speed v Blackburn at St James in 2001/02 ...,en,NEUTRAL,"{""Neutral"":0.998256266117095947265625,""Negativ..."
2,2018-09-05T12:54:20.408000,1037323043360657408,@ChelseaFC Don't make him regret it and start ...,en,NEUTRAL,"{""Neutral"":0.912796199321746826171875,""Negativ..."
3,2018-05-08T10:42:17,993803266323550208,"@LiverpoolFF @AnfieldEdition He's a liar, made...",en,NEGATIVE,"{""Neutral"":0.3271420896053314208984375,""Negati..."
4,2018-08-07T07:29:59.136000,1026732168226267136,@theesk @Everton Didn't realise Kenwright is d...,en,NEUTRAL,"{""Neutral"":0.957906246185302734375,""Negative"":..."


In [2]:
tweets = tweets.drop(['tweet_date_created'], axis=1)
tweets.head(5)

duplicates = tweets[tweets.duplicated(subset=['tweet_id'], keep=False)]

if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate tweet ids. Removing duplicates...")
    tweets.drop_duplicates(subset=['tweet_id'], inplace=True)
else:
    print("No duplicate tweet ids found.")

tweets = tweets.drop(['sentiment_score'], axis=1)

Found 762643 duplicate tweet ids. Removing duplicates...


In [3]:
tweets = tweets.dropna()
print(tweets.isnull().sum())

tweet_id      0
tweet_text    0
language      0
sentiment     0
dtype: int64


In [4]:
all_english = (tweets['language'] == 'en').all()

if all_english:
    print("All values in the 'language' column are 'en'")
else:
    print("Not all values in the 'language' column are 'en'")

All values in the 'language' column are 'en'


In [5]:
tweets = tweets.drop(['language'], axis=1)
tweets.head(5)

,tweet_id,tweet_text,sentiment
0,993767246437666816,Bayer Leverkusen goalkeeper Bernd Leno will no...,NEUTRAL
1,1013866900772835331,Gary Speed v Blackburn at St James in 2001/02 ...,NEUTRAL
2,1037323043360657408,@ChelseaFC Don't make him regret it and start ...,NEUTRAL
3,993803266323550208,"@LiverpoolFF @AnfieldEdition He's a liar, made...",NEGATIVE
4,1026732168226267136,@theesk @Everton Didn't realise Kenwright is d...,NEUTRAL


In [6]:
tweets = tweets.drop(['tweet_id'], axis=1)

In [7]:
# Get all unique sentiment values in the 'sentiment' column
unique_sentiments = tweets['sentiment'].unique()

# Print the unique sentiment values
print("Unique sentiment values in the 'sentiment' column:")
for sentiment in unique_sentiments:
    print(sentiment)

Unique sentiment values in the 'sentiment' column:
NEUTRAL
NEGATIVE
POSITIVE
MIXED


In [8]:
# Define a dictionary that maps sentiment labels to numeric values
sentiment_map = {"NEUTRAL": 0, "POSITIVE": 1, "MIXED": 3, "NEGATIVE": 2}

# Map the sentiment labels to their numeric values
tweets['sentiment_values'] = tweets['sentiment'].map(sentiment_map)

# Print the new column that contains the mapped values
print(tweets['sentiment_values'])

0          0
1          0
2          0
3          2
4          0
          ..
5393957    0
5393958    2
5393959    0
5393960    0
5393961    0
Name: sentiment_values, Length: 5012534, dtype: int64


In [9]:
tweets = tweets.drop(['sentiment'], axis=1)
tweets.head(5)

,tweet_text,sentiment_values
0,Bayer Leverkusen goalkeeper Bernd Leno will no...,0
1,Gary Speed v Blackburn at St James in 2001/02 ...,0
2,@ChelseaFC Don't make him regret it and start ...,0
3,"@LiverpoolFF @AnfieldEdition He's a liar, made...",2
4,@theesk @Everton Didn't realise Kenwright is d...,0


In [10]:
# Find the indices of rows with Mixed sentiment
mixed_indices = tweets[tweets['sentiment_values'] == 3].index

# Delete the rows with Mixed sentiment
tweets = tweets.drop(mixed_indices)

# Print the resulting DataFrame
print(tweets.head())

                                          tweet_text  sentiment_values
0  Bayer Leverkusen goalkeeper Bernd Leno will no...                 0
1  Gary Speed v Blackburn at St James in 2001/02 ...                 0
2  @ChelseaFC Don't make him regret it and start ...                 0
3  @LiverpoolFF @AnfieldEdition He's a liar, made...                 2
4  @theesk @Everton Didn't realise Kenwright is d...                 0


In [11]:
positive_tweets = tweets[tweets['sentiment_values'] == 1]
negative_tweets = tweets[tweets['sentiment_values'] == 2]
neutral_tweets = tweets[tweets['sentiment_values'] == 0]

print('No of positive tagged tweets is: {}'.format(len(positive_tweets)))
print('No of negative tagged tweets is: {}'.format(len(negative_tweets)))
print('No of neutral tagged tweets is: {}'.format(len(neutral_tweets)))

No of positive tagged tweets is: 1070334
No of negative tagged tweets is: 354501
No of neutral tagged tweets is: 3549918


In [12]:
import pandas as pd

# Shuffle the DataFrame to ensure that the downsampling is random
tweets = tweets.sample(frac=1, random_state=42)

# Count the number of tweets in each sentiment class
counts = tweets['sentiment_values'].value_counts()

# Find the smallest class size
smallest_size = counts.min()

# Downsample each class to the smallest size
positive_tweets = tweets[tweets['sentiment_values'] == 1].sample(n=smallest_size, random_state=42)
negative_tweets = tweets[tweets['sentiment_values'] == 2].sample(n=smallest_size, random_state=42)
neutral_tweets = tweets[tweets['sentiment_values'] == 0].sample(n=smallest_size, random_state=42)

# Concatenate the downsampled DataFrames
tweets = pd.concat([positive_tweets, negative_tweets, neutral_tweets], ignore_index=True)

# Print the new counts of tweets in each class
print('No of positive tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 1])))
print('No of negative tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 2])))
print('No of neutral tagged tweets is: {}'.format(len(tweets[tweets['sentiment_values'] == 0])))


No of positive tagged tweets is: 354501
No of negative tagged tweets is: 354501
No of neutral tagged tweets is: 354501


In [13]:
tweets['tweet_text'] = tweets['tweet_text'].astype('str')

In [14]:
import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\clayt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle

urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
some = 'amp|today|tomorrow|going|girl'

def process_tweets(tweet):
  # Lower Casing
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)  
    
    tweet = re.sub(r"some1", "someone", tweet)
    tweet = re.sub(r"yrs", "years", tweet)
    tweet = re.sub(r"hrs", "hours", tweet)
    tweet = re.sub(r"2morow|2moro", "tomorrow", tweet)
    tweet = re.sub(r"2day", "today", tweet)
    tweet = re.sub(r"4got|4gotten", "forget", tweet)
    tweet = re.sub(r"b-day|bday", "b-day", tweet)
    tweet = re.sub(r"mother's", "mother", tweet)
    tweet = re.sub(r"mom's", "mom", tweet)
    tweet = re.sub(r"dad's", "dad", tweet)
    tweet = re.sub(r"hahah|hahaha|hahahaha", "haha", tweet)
    tweet = re.sub(r"lmao|lolz|rofl", "lol", tweet)
    tweet = re.sub(r"thanx|thnx", "thanks", tweet)
    tweet = re.sub(r"goood", "good", tweet)
    tweet = re.sub(r"some1", "someone", tweet)
    tweet = re.sub(r"some1", "someone", tweet)
    tweet = tweet.lower()
    tweet=tweet[1:]

    # Removing all URls 
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet) 
    #remove some words
    tweet= re.sub(some,'',tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #tokens = [w for w in tokens if len(w)>2]
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem 
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [16]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", 
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
     "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [17]:
def convert_abbrev_in_text(tweets):
    t = []
    words = tweets.split() 
    t = [abbreviations[w.lower()] if w.lower() in abbreviations.keys() else w for w in words]

In [18]:
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    return text

In [19]:
import pandas as pd
from sklearn.utils import shuffle

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['tweet_text'].apply(preprocess_text)

tweets =  shuffle(tweets).reset_index(drop=True)

In [20]:
tokenized_tweet=tweets['processed_text'].apply(lambda x: x.split())
tokenized_tweet.head(5)

0           [well, done, trent, hard, work, pays, off]
1    [definitely, some, players, in, the, under, 23...
2    [you, cant, blame, mourinho, for, shit, defend...
3    [these, united, fans, are, a, disgrace, weve, ...
4    [woodward, and, the, glazers, leveraging, the,...
Name: processed_text, dtype: object

In [21]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [22]:
X = tweets['processed_text']
y = tweets['sentiment_values']

In [23]:
import pandas as pd

# Assuming your DataFrame is named 'df'
total_rows = tweets.shape[0]
print("Total number of rows:", total_rows)


Total number of rows: 1063503


In [24]:
import numpy as np

tweet_lengths = tweets['processed_text'].apply(lambda x: len(x.split()))
mean_length = np.mean(tweet_lengths)
std_length = np.std(tweet_lengths)

print("Average tweet length:", mean_length)
print("Standard deviation of tweet length:", std_length)


Average tweet length: 19.881952378131516
Standard deviation of tweet length: 12.338064194346824


In [25]:
max_features = 30000
max_length = 100

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=max_length)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

CNN and LSTM Implementation

ChatGPT Model - GloVe Implementation

In [31]:
import numpy as np
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

# Load GloVe embeddings
def load_glove_embeddings(file_path, embedding_dim, word_index, max_features):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((max_features, embedding_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Load the GloVe embeddings matrix
glove_file_path = 'glove.twitter.27B.200d.txt'  # Update the path to the downloaded GloVe file
embedding_dim = 200
embedding_matrix = load_glove_embeddings(glove_file_path, embedding_dim, tokenizer.word_index, max_features)

# Create the model with GloVe embeddings
def create_lstm_model(embedding_matrix, embed_dim=200, lstm_out=256, dropout_rate=0.2, optimizer='adam', num_classes=3):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, weights=[embedding_matrix], input_length=X.shape[1], trainable=False))
    model.add(Bidirectional(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create and train the model with GloVe embeddings
model = create_lstm_model(embedding_matrix)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_lstm_model.h5', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(X_train, y_train, epochs=5, batch_size=500, validation_split=0.1, callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

# Predict sentiment labels for the test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate the accuracy of the predictions
correct_predictions = np.sum(y_pred == y_test)
total_predictions = len(y_test)
prediction_accuracy = correct_predictions / total_predictions

print(f"Prediction accuracy: {prediction_accuracy:.4f}")


Epoch 1/5
1341/1341 [==============================] - 9214s 7s/step - loss: 0.5114 - accuracy: 0.7880 - val_loss: 0.4378 - val_accuracy: 0.8209
Epoch 2/5
1341/1341 [==============================] - 12235s 9s/step - loss: 0.4364 - accuracy: 0.8220 - val_loss: 0.4075 - val_accuracy: 0.8353
Epoch 3/5
1341/1341 [==============================] - 10670s 8s/step - loss: 0.4120 - accuracy: 0.8324 - val_loss: 0.4110 - val_accuracy: 0.8340
Epoch 4/5
1341/1341 [==============================] - 10016s 7s/step - loss: 0.3958 - accuracy: 0.8395 - val_loss: 0.3944 - val_accuracy: 0.8411
Epoch 5/5
9971/9971 [==============================] - 1793s 180ms/step - loss: 0.3920 - accuracy: 0.8399
Test accuracy: 0.8399347066879272
9971/9971 [==============================] - 1497s 150ms/step
Prediction accuracy: 0.8399


In [32]:
model.save("lstm_sentiment_modegov.h5")

The parameters using this: dropout rate of 0.2, Adam optimizer, and sparse categorical cross-entropy loss function achieved the best performance
during training, with a batch size of 500.

In [ ]:
from numba import jit, cuda
from keras.utils import plot_model

def create_lstm_model(embed_dim=192, lstm_out=256, dropout_rate=0.2, optimizer='adam', num_classes=3):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(Bidirectional(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Make sure your target variable 'y' has integer labels
# If it's not the case, you should convert it accordingly

# Create and train the model with specified parameters
model = create_lstm_model()
history = model.fit(X_train, y_train, epochs=3, batch_size=500, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

#plot_model(model, to_file='best_lstm_model.png', show_shapes=True)

Epoch 1/3
   6/1341 [..............................] - ETA: 13:10:37 - loss: 1.0951 - accuracy: 0.3680

KeyboardInterrupt: 

In [ ]:
# from kerastuner import HyperModel
# from kerastuner.tuners import RandomSearch

# class LSTMHyperModel(HyperModel):
#     def __init__(self, input_length, num_classes):
#         self.input_length = input_length
#         self.num_classes = num_classes

#     def build(self, hp):
#         model = Sequential()
#         model.add(Embedding(max_features,
#                             hp.Int('embed_dim', min_value=64, max_value=256, step=32),
#                             input_length=self.input_length))
#         model.add(Bidirectional(LSTM(hp.Int('lstm_out', min_value=128, max_value=256, step=32),
#                                      dropout=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1),
#                                      recurrent_dropout=hp.Float('recurrent_dropout_rate', min_value=0.1, max_value=0.5, step=0.1))))
#         model.add(Dense(self.num_classes, activation='softmax'))
#         model.compile(loss='sparse_categorical_crossentropy',
#                       optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
#                       metrics=['accuracy'])
#         return model

Modified model to use Grid Search instead

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, LSTM, Bidirectional
# from keras.optimizers import Adam, RMSprop
# from keras.utils import plot_model

# input_length = X.shape[1]
# num_classes = 3

# def create_lstm_model(embed_dim=64, lstm_out=128, dropout_rate=0.1, recurrent_dropout_rate=0.1, optimizer='adam'):
#     model = Sequential()
#     model.add(Embedding(max_features, embed_dim, input_length=input_length))
#     model.add(Bidirectional(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate)))
#     model.add(Dense(num_classes, activation='softmax'))
#     model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#     return model

# # Define the hyperparameters to search over
# hyperparameters = {
#     'embed_dim': [64, 96, 128, 160, 192, 224, 256],
#     'lstm_out': [128, 160, 192, 224, 256],
#     'dropout_rate': [0.1, 0.2, 0.3, 0.4],
#     'recurrent_dropout_rate': [0.1, 0.2, 0.3, 0.4],
#     'optimizer': [Adam(), RMSprop()]
# }

# # Create the Keras classifier
# lstm_model = KerasClassifier(build_fn=create_lstm_model, epochs=3, batch_size=500, verbose=0)

# # Initialize the GridSearchCV object
# grid_search = GridSearchCV(estimator=lstm_model, param_grid=hyperparameters, scoring='accuracy', cv=5)

# # Perform the hyperparameter search
# grid_search.fit(X_train, y_train)

# # Print the results
# print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))

# # Save the best model
# best_model = grid_search.best_estimator_.model
# best_model.save('best_lstm_model.h5')

# # Evaluate the best model on the test set
# loss, accuracy = best_model.evaluate(X_test, y_test)
# print("Test accuracy:", accuracy)

# # Print the hyperparameters and the associated validation accuracy for each trial
# for i, params in enumerate(grid_search.cv_results_['params']):
#     print("Trial ID:", i)
#     print("Hyperparameters:")
#     for key, value in params.items():
#         print(f"{key}: {value}")
#     print("Validation Accuracy:", grid_search.cv_results_['mean_test_score'][i])
#     print("\n")

# print(grid_search.cv_results_['params'])
# print(grid_search.cv_results_['mean_test_score'])
# print(grid_search.cv_results_['std_test_score'])

# plot_model(best_model, to_file='best_lstm_model.png', show_shapes=True)

4653/4653 [==============================] - 813s 175ms/step


KeyboardInterrupt: 

In [ ]:
# # Save the model to a file
model.save("lstm_sentiment_model2.h5")